In [1]:
import PIL
from torchvision import transforms

def get_test_img(file):
    test_img = PIL.Image.open(f'test pictures\\{file}')
    return transforms.ToTensor()(test_img)

In [3]:
import torch
import os
from models.SR3Builder import SR3Builder
from models.SRDIFFBuilder import SRDiffBuilder
from utils.model_utils import load_model

sr3_models = os.listdir('saved models\\SRDiff')
device = torch.device(0)
model_builder = SRDiffBuilder()
model = model_builder.set_standart().build()
model.to(device)
sr_images = []
for sr3_model in sr3_models:
    torch.cuda.empty_cache()
    model = load_model(model, sr3_model, 'saved models\\SRDiff\\')
    bicubic = get_test_img("Amnesty POI-6-3-1.png").unsqueeze(0).to(device)
    lr = get_test_img("Amnesty POI-6-3-1-lr.png").unsqueeze(0).to(device)    
    sr, _ = model.sample(lr, bicubic, (1,3, 256, 256), True)
    torch.cuda.empty_cache()
    sr_images.append(sr.squeeze().to('cpu').detach())

The model weights have been loaded from 'saved models\SRDiff\SRDiff Standart 128 bach.pt'
The model weights have been loaded from 'saved models\SRDiff\SRDiff Standart 20 bach 100 ep DA rrdbfix.pt'
The model weights have been loaded from 'saved models\SRDiff\SRDiff Standart 20 bach 20 ep DA rrdbfix.pt'
The model weights have been loaded from 'saved models\SRDiff\SRDiff Standart 20 bach 20 ep DA.pt'
The model weights have been loaded from 'saved models\SRDiff\SRDiff Standart 20 bach 50 ep DA.pt'
The model weights have been loaded from 'saved models\SRDiff\SRDiff Standart 20 bach 50 ep.pt'
The model weights have been loaded from 'saved models\SRDiff\SRDiff Standart 20 bach.pt'


In [4]:
from utils.tensor_utils import tensor2img

for idx, tensor in enumerate(sr_images):
    img_np = tensor2img(tensor)
    img = PIL.Image.fromarray(img_np)
    img.save(f"test pictures\\SRDiff\\{sr3_models[idx]}.png")